In [1]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)

from main import *

c:\Users\SSosa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from numpyro.infer import Predictive, SVI, Trace_ELBO

In [10]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)

from main import bi
import time as tm
# setup platform------------------------------------------------
m = bi(platform='cpu')

# import data ------------------------------------------------
m.data('../resources/data/Howell1.csv', sep=';') 
m.df = m.df[m.df.age > 18]
m.scale(['weight'])
#m.data_to_model(['weight', 'height'])

# define model ------------------------------------------------
def model(weight, height):    
    a = m.dist.normal( 178, 20, name = 'a')
    b = m.dist.lognormal(  0, 1, name = 'b')   
    s = m.dist.uniform( 0, 50, name = 's')
    m.lk.normal(a + b * weight , s, obs=height)
     #numpyro.sample('name', numpyro.distributions.Normal(a + b * weight , s),height)

# Run sampler ------------------------------------------------


jax.local_device_count 16


In [4]:
# FIT DP SB GMM via ADVI
sigmoid = lambda x: 1 / (1 + onp.exp(-x))


In [5]:
import json
import matplotlib.pyplot as plt
from jax import random, lax
import jax.numpy as np
from jax.scipy.special import logsumexp
import numpyro
import numpyro.distributions as dist
from numpyro.distributions import constraints
from numpyro.infer.autoguide import AutoDiagonalNormal
from numpyro.infer import SVI, ELBO
from numpyro.infer import MCMC, NUTS, HMC
import numpy as onp

In [12]:

# Setup ADVI.
guide = AutoDiagonalNormal(model)  # surrogate posterior
optimizer = numpyro.optim.Adam(step_size=0.01)  # adam optimizer
svi = SVI(guide.model, guide, optimizer, loss=ELBO())  # ELBO loss
init_state = svi.init(random.PRNGKey(2), m.df.height.values, 10)  # initial state

# Run optimizer
state, losses = lax.scan(lambda state, i: 
                         svi.update(state, m.df.height.values, 10), init_state, np.arange(2000))

NotImplementedError: This ELBO objective does not support mutable state.